In [1]:
import math
import ast
import heapq
from copy import deepcopy
from math import log
import random
import os
import copy
import time
import requests
from io import BytesIO

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
from torchvision import transforms

from PIL import Image

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.ion()

In [2]:
if torch.cuda.is_available():
    print("Using CUDA")
    device = torch.device("cuda")
else:
    print("Using CPU")
    device = torch.device("cpu")

Using CUDA


In [3]:
start_token = '<start>'
end_token = '<end>'
unknown_token = '<unk>'
padding_token = '<pad>'

## Tools

In [4]:
def split_tensor1d(tensor, split):
    r"""Split 1D of tensor into N split

    Arguments
        tensor (Pytorch.Tensor) : tensor to split
        split  (int) : number of split
    Return
        array of splitted tensor with N elements of array
    """
    return [
        tensor[:split],
        tensor[split: split * 2],
        tensor[split * 2: split * 3],
        tensor[split * 3:],
    ]


def split_tensor2d(tensor, split, front=False):
    r"""Split 2D of tensor into N split of 2D tensor

    Arguments
        tensor (Pytorch.Tensor) : tensor to split
        split  (int) : number of split
        front  (bool) : split axis 0 if True else axis 1
    Return
        array of splitted 2D tensor with N elements of array
    """

    if front:
        return [
            tensor[:split, :],
            tensor[split: split * 2, :],
            tensor[split * 2: split * 3, :],
            tensor[split * 3:, :],
        ]

    return [
        tensor[:, :split],
        tensor[:, split: split * 2],
        tensor[:, split * 2: split * 3],
        tensor[:, split * 3:],
    ]


In [5]:
def indexto1hot(vocab_len, index):
    #print("index type: ")
    if isinstance(index, int) == False:
        n = len(index)
        #print("making a 1hot encoding of shape: " + str(n) + "," + str(vocab_len) )
        one_hot = np.zeros([n,vocab_len])
        #can this be optimized?
        for i in range(n):
            one_hot[i,index[i]]=1
        
        return one_hot
    else:
        one_hot = np.zeros([vocab_len])
        one_hot[index] = 1
        
        return one_hot

In [6]:
def make_zeros(shape, cuda=False):
    zeros = torch.zeros(shape)
    if cuda:
        zeros = zeros.cuda()
    return zeros

## Class Definition

In [7]:
class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def get_word(self, idx):
        return self.idx2word[idx]

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)


In [8]:
class AttentionLSTM(nn.Module):
    def __init__(self, dim_word_emb, dim_lang_lstm, dim_image_feats, nb_hidden):
        super(AttentionLSTM, self).__init__()
        
        self.lstm_cell = nn.LSTMCell(dim_lang_lstm+dim_image_feats+dim_word_emb,
                                     nb_hidden,
                                     bias=True)
        
    def forward(self, h1, c1, h2, v_mean, word_emb):
        #print(h2.shape)
        #print(v_mean.shape)
        #print(word_emb.shape)
        input_feats = torch.cat((h2, v_mean, word_emb),dim=1)
        h_out, c_out = self.lstm_cell(input_feats, (h1, c1))
        
        return h_out, c_out


In [9]:
class Attend(nn.Module):
    def __init__(self, dim_image_feats, dim_att_lstm, nb_hidden):
        super(Attend, self).__init__()
    
        self.fc_image_feats = nn.Linear(dim_image_feats, nb_hidden, bias=False)
        self.fc_att_lstm = nn.Linear(dim_att_lstm, nb_hidden, bias=False)
        self.act_tan = nn.Tanh()
        self.fc_att = nn.Linear(nb_hidden, 1)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, image_feats, h1):
        nb_batch, nb_feats, feat_dim = image_feats.size()
        att_lstm_emb = self.fc_att_lstm(h1).unsqueeze(1)
        image_feats_emb = self.fc_image_feats(image_feats)
        all_feats_emb = image_feats_emb + att_lstm_emb.repeat(1,nb_feats,1)

        activate_feats = self.act_tan(all_feats_emb)
        unnorm_attention = self.fc_att(activate_feats)
        normed_attention = self.softmax(unnorm_attention)

        #print(normed_attention.shape)
        #print(nb_feats)
        #print(image_feats.shape)
        #weighted_feats = normed_attention.repeat(1,1,nb_feats) * image_feats
        weighted_feats = normed_attention * image_feats
        #print(weighted_feats.shape)
        attended_image_feats = weighted_feats.sum(dim=1)
        #print(attended_image_feats.shape)
    
        return attended_image_feats


In [10]:
class SCNCell(nn.Module):
    r"""Custom Cell for Implementing Semantic Compositional Networks

    Arguments
        input_size (int): size of input
        hidden_size (int): size of embedding
        semantic_size (int): size of semantic
        factor_size (int): size of factor
        bias (boolean, optional): use bias?
    """

    def __init__(self, input_size, hidden_size, semantic_size, factor_size, bias=True):
        super(SCNCell, self).__init__()

        self.factor_size = factor_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.semantic_size = semantic_size

        self.weight_ia = nn.Parameter(
            torch.Tensor(input_size, 4 * factor_size))
        self.weight_ib = nn.Parameter(
            torch.Tensor(semantic_size, 4 * factor_size))
        self.weight_ic = nn.Parameter(
            torch.Tensor(hidden_size, 4 * factor_size))

        self.weight_ha = nn.Parameter(
            torch.Tensor(hidden_size, 4 * factor_size))
        self.weight_hb = nn.Parameter(
            torch.Tensor(semantic_size, 4 * factor_size))
        self.weight_hc = nn.Parameter(
            torch.Tensor(hidden_size, 4 * factor_size))

        if bias:
            self.bias_ih = nn.Parameter(torch.Tensor(4 * hidden_size))
            self.bias_hh = nn.Parameter(torch.Tensor(4 * hidden_size))
        else:
            self.register_parameter('bias_ih', None)
            self.register_parameter('bias_hh', None)

        self.reset_parameters()

    def forward(self, h1, v_hat, semantic_input, hx=None):
        r"""Forward propagation.

        Arguments
            wemb_input (torch.Tensor): word embedding input, a tensor of dimension (batch_size, input_size)
            semantic_input (torch.Tensor): semantic concepts input, a tensor of dimension (batch_size, semantic_dim)
        Returns
            torch.Tensor: next hidden state, next cell state
        """

        input_feats = torch.cat((h1, v_hat), dim=1)
        
        self.check_forward_input(input_feats)

        [ia_i, ia_f, ia_o, ia_c] = split_tensor2d(
            self.weight_ia, self.factor_size)
        [ib_i, ib_f, ib_o, ib_c] = split_tensor2d(
            self.weight_ib, self.factor_size)
        [ic_i, ic_f, ic_o, ic_c] = split_tensor2d(
            self.weight_ic, self.factor_size)
        [b_ii, b_if, b_io, b_ic] = split_tensor1d(
            self.bias_ih, self.hidden_size)

        tmp1_i = (input_feats @ ia_i)
        tmp1_f = (input_feats @ ia_f)
        tmp1_o = (input_feats @ ia_o)
        tmp1_c = (input_feats @ ia_c)

        tmp2_i = (semantic_input @ ib_i).unsqueeze(0)
        tmp2_f = (semantic_input @ ib_f).unsqueeze(0)
        tmp2_o = (semantic_input @ ib_o).unsqueeze(0)
        tmp2_c = (semantic_input @ ib_c).unsqueeze(0)

        state_below_i = ((tmp1_i * tmp2_i) @ ic_i.t()) + b_ii
        state_below_f = ((tmp1_f * tmp2_f) @ ic_f.t()) + b_if
        state_below_o = ((tmp1_o * tmp2_o) @ ic_o.t()) + b_io
        state_below_c = ((tmp1_c * tmp2_c) @ ic_c.t()) + b_ic

        x_i = state_below_i.squeeze(0)
        x_f = state_below_f.squeeze(0)
        x_o = state_below_o.squeeze(0)
        x_c = state_below_c.squeeze(0)

        if hx is None:
            hx = input_feats.new_zeros(input_feats.size(
                0), self.hidden_size, requires_grad=False)
            hx = (hx, hx)

        self.check_forward_hidden(x_i, hx[0], '[0]')
        self.check_forward_hidden(x_i, hx[1], '[1]')

        self.check_forward_hidden(x_f, hx[0], '[0]')
        self.check_forward_hidden(x_f, hx[1], '[1]')

        self.check_forward_hidden(x_o, hx[0], '[0]')
        self.check_forward_hidden(x_o, hx[1], '[1]')

        self.check_forward_hidden(x_c, hx[0], '[0]')
        self.check_forward_hidden(x_c, hx[1], '[1]')

        return self.recurrent_step(x_i, x_f, x_o, x_c, semantic_input, hx)

    def recurrent_step(self, x_i, x_f, x_o, x_c, semantic_input, hx):
        r"""Recurrent step helper for forward propagation.

        Arguments
            x_i, x_f, x_o, x_c (torch.Tensor): factorized input, containing information from word embedding and semantic concepts, tensors of dimension (batch_size, input_size)
            semantic_input (torch.Tensor): semantic concepts input, a tensor of dimension (batch_size, semantic_dim)
            h_x (torch.Tensor): initial value of hidden and cell state
        Returns
            torch.Tensor: next hidden state, next cell state
        """

        h_, c_ = hx

        [ha_i, ha_f, ha_o, ha_c] = split_tensor2d(
            self.weight_ha, self.factor_size)
        [hb_i, hb_f, hb_o, hb_c] = split_tensor2d(
            self.weight_hb, self.factor_size)
        [hc_i, hc_f, hc_o, hc_c] = split_tensor2d(
            self.weight_hc, self.factor_size)
        [b_hi, b_hf, b_ho, b_hc] = split_tensor1d(
            self.bias_hh, self.hidden_size)

        preact_i = (h_ @ ha_i) * (semantic_input @ hb_i)
        preact_i = (preact_i @ hc_i.t()) + x_i + b_hi

        preact_f = (h_ @ ha_f) * (semantic_input @ hb_f)
        preact_f = (preact_f @ hc_f.t()) + x_f + b_hf

        preact_o = (h_ @ ha_o) * (semantic_input @ hb_o)
        preact_o = (preact_o @ hc_o.t()) + x_o + b_ho

        preact_c = (h_ @ ha_c) * (semantic_input @ hb_c)
        preact_c = (preact_c @ hc_c.t()) + x_c + b_hc

        i = torch.sigmoid(preact_i)
        f = torch.sigmoid(preact_f)
        o = torch.sigmoid(preact_o)
        c = torch.tanh(preact_c)

        c = f * c_ + i * c
        h = o * torch.tanh(c)

        return h, c

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            torch.nn.init.uniform_(weight, -stdv, stdv)

    def extra_repr(self):
        s = '{input_size}, {hidden_size}'
        if 'bias' in self.__dict__ and self.bias is not True:
            s += ', bias={bias}'
        if 'nonlinearity' in self.__dict__ and self.nonlinearity != "tanh":
            s += ', nonlinearity={nonlinearity}'
        return s.format(**self.__dict__)

    def check_forward_input(self, input):
        if input.size(1) != self.input_size:
            raise RuntimeError(
                "input has inconsistent input_size: got {}, expected {}".format(
                    input.size(1), self.input_size))

    def check_forward_hidden(self, input, hx, hidden_label=''):
        if input.size(0) != hx.size(0):
            raise RuntimeError(
                "Input batch size {} doesn't match hidden{} batch size {}".format(
                    input.size(0), hidden_label, hx.size(0)))

        if hx.size(1) != self.hidden_size:
            raise RuntimeError(
                "hidden{} has inconsistent hidden_size: got {}, expected {}".format(
                    hidden_label, hx.size(1), self.hidden_size))


In [11]:
# Sentence is class for beam search
class Sentence(object):
    def __init__(self, max_nb_words, beam_width, end_word, vocab):
        self.max_nb_words = max_nb_words
        self.beam_width = beam_width
        self.end_word = end_word
        self.vocab = vocab

        self.words = []
        self.probability = 0
        self.ended = False

        self.act = nn.Softmax(dim=1)

    def update_words(self, s, state, y):
        y = self.act(y)
        new_s = []
        for i in range(self.beam_width):
            val, idx = y.max(dim=1)
            y[0, idx] -= val
            current_word = y.clone()
            current_word[0,:] = 0
            current_word[0,idx] = 1
            s2 = s.copy()
            s2.update_state(val, state, current_word)
            new_s.append(s2)
            if s2.ended:
                break
        return new_s

    def update_state(self, p, state, current_word):
        self.state = [s.clone() for s in state]
        self.words.append(current_word)
        self._update_probability(p)
        self._update_finished()

    def get_states(self):
        return self.state, self.words[-1]

    def extract_sentence(self):
        sentence = []
        for w in self.words:
            idx = w.max(1)[1].item()
            sentence.append(self.vocab.get_word(idx))
        return [self.probability, sentence]

    def _update_probability(self, p):
        self.probability += log(p, 2)

    def _update_finished(self):
        n = len(self.words)
        f = self.words[-1]
        if (n > self.max_nb_words) or (f == self.end_word).all():
            self.ended = True

    def copy(self):
        new = Sentence(self.max_nb_words,
                       self.beam_width,
                       self.end_word,
                       self.vocab)
        new.words = [w.clone() for w in self.words]
        new.probability = self.probability
        return new

    def __lt__(self, other):
        return self.probability < other.probability

    def __repr__(self):
        s = ''
        for w in self.words:
            idx = w.max(1)[1].item()
            s += "{}, ".format(self.vocab.get_word(idx))
        return s


class Beam(object):
    def __init__(self, beam_width):
        self.beam_width = beam_width
        self.heap = []

    def push(self, s):
        s.probability *= -1
        heapq.heappush(self.heap, s)

    def pop(self):
        s = heapq.heappop(self.heap)
        s.probability *= -1
        return s

    def trim(self):
        h2 = []
        for i in range(self.beam_width):
            if len(self.heap) == 0:
                break
            heapq.heappush(h2, heapq.heappop(self.heap))
        self.heap=h2

    def __len__(self):
        return len(self.heap)


In [12]:
class PredictWord(nn.Module):
    def __init__(self, dim_language_lstm, dict_size):
        super(PredictWord, self).__init__()
        self.fc = nn.Linear(dim_language_lstm, dict_size)
        
    def forward(self, h2):
        y = self.fc(h2)
        return y

In [13]:
class ImageTagger(nn.Module):
    r"""Tagger Encoder extends ResNet152 Model.

    Arguments:
        semantic_size (int, optional): size of semantic size
        dropout (float, optional): dropout rate
    """

    def __init__(self, semantic_size=1000, dropout=0.15):
        super(ImageTagger, self).__init__()
        self.semantic_size = semantic_size

        # Using pre-trained ImageNet
        resnet = torchvision.models.resnet152(pretrained=True)

        # Remove linear layers (since we're not doing classification)
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)

        self.dropout = nn.Dropout(dropout)

        self.linear = nn.Linear(2048, semantic_size)

        self.sigmoid = nn.Sigmoid()

        self.fine_tune()

    def forward(self, images):
        r"""Forward propagation.

        Arguments
            images (torch.Tensor): images, a tensor of dimensions (batch_size, 3, image_size, image_size)
        Returns 
            torch.Tensor: probabilites of tags (batch_size, 1000)
        """
        out = self.resnet(images)
        out = out.view(out.size(0), -1)   # (batch_size, 2048)
        out = self.dropout(out)           # (batch_size, 2048)
        out = self.linear(out)            # (batch_size, 1000)
        out = self.sigmoid(out)           # (batch_size, 1000)
        
        return out

    def fine_tune(self, fine_tune=True):
        r"""Allow or prevent the computation of gradients for convolutional blocks 2 through 4 of the encoder.

        Arguments
            fine_tune (boolean): Allow fine tuning?
        """
        for p in self.resnet.parameters():
            p.requires_grad = False
        # If fine-tuning, only fine-tune convolutional blocks 2 through 4
        for c in list(self.resnet.children())[5:]:
            for p in c.parameters():
                p.requires_grad = fine_tune

In [14]:
class CaptionModel(nn.Module):

    def __init__(self, dict_size, image_feature_dim, vocab, tf_ratio):
        super(CaptionModel, self).__init__()
        self.dict_size = dict_size
        self.image_feature_dim = image_feature_dim
        self.vocab = vocab
        self.tf_ratio = tf_ratio

        self.embed_word = nn.Linear(dict_size, WORD_EMB_DIM, bias=False)
        self.lstm1 = AttentionLSTM(WORD_EMB_DIM,
                                   NB_HIDDEN_LSTM2,
                                   image_feature_dim,
                                   NB_HIDDEN_LSTM1)
        self.lstm2 = SCNCell(NB_HIDDEN_LSTM1+image_feature_dim,
                             NB_HIDDEN_LSTM2,
                             SEMANTIC_DIM,
                             FACTORED_DIM,
                             bias=True)
        self.attention = Attend(image_feature_dim,
                                NB_HIDDEN_LSTM1,
                                NB_HIDDEN_ATT)
        self.predict_word = PredictWord(NB_HIDDEN_LSTM2, dict_size)

        self.h1 = torch.nn.Parameter(torch.zeros(1, NB_HIDDEN_LSTM1))
        self.c1 = torch.nn.Parameter(torch.zeros(1, NB_HIDDEN_LSTM1))
        self.h2 = torch.nn.Parameter(torch.zeros(1, NB_HIDDEN_LSTM2))
        self.c2 = torch.nn.Parameter(torch.zeros(1, NB_HIDDEN_LSTM2))
    
    def forward(self, image_feats, nb_timesteps, semantic_input, true_words, beam=None):
        if beam is not None:
            return self.beam_search(image_feats, nb_timesteps, beam, semantic_input)

        nb_batch, nb_image_feats, _ = image_feats.size()
        use_cuda = image_feats.is_cuda

        v_mean = image_feats.mean(dim=1)

        state, current_word = self.init_inference(nb_batch, use_cuda)
        y_out = make_zeros((nb_batch, nb_timesteps-1, self.dict_size), cuda = use_cuda)

        for t in range(nb_timesteps-1):
            y, state = self.forward_one_step(state,
                                             current_word,
                                             v_mean,
                                             image_feats,
                                             semantic_input)
            y_out[:,t,:] = y

            current_word = self.update_current_word(y, true_words, t, use_cuda)

        return y_out

    def forward_one_step(self, state, current_word, v_mean, image_feats, semantic_input):
        h1, c1, h2, c2 = state
        word_emb = self.embed_word(current_word)
        h1, c1 = self.lstm1(h1, c1, h2, v_mean, word_emb)
        v_hat = self.attention(image_feats, h1)
        h2, c2 = self.lstm2(h1, v_hat, semantic_input)
        y = self.predict_word(h2)
        state = [h1, c1, h2, c2]
        
        return y, state

    def update_current_word(self, y, true_words, t, cuda):
        use_tf = True if random.random() < self.tf_ratio else False
        if use_tf:
            next_word = true_words[:,t+1]
        else:
            next_word = torch.argmax(y, dim=1)

        current_word = indexto1hot(len(self.vocab), next_word)
        current_word = torch.from_numpy(current_word).float()
        
        if cuda:
            current_word = current_word.cuda()
    
        return current_word

    def init_inference(self, nb_batch, cuda):
        start_word = indexto1hot(len(self.vocab), self.vocab('<start>'))
        start_word = torch.from_numpy(start_word).float().unsqueeze(0)
        start_word = start_word.repeat(nb_batch, 1)

        if cuda:
            start_word = start_word.cuda()

        h1 = self.h1.repeat(nb_batch, 1)
        c1 = self.c1.repeat(nb_batch, 1)
        h2 = self.h2.repeat(nb_batch, 1)
        c2 = self.c2.repeat(nb_batch, 1)
        state = [h1, c1, h2, c2]

        return state, start_word

    #########################################
    #               BEAM SEARCH             #
    #########################################
    def beam_search(self, image_features, max_nb_words, beam_width, semantic_input):
        # Initialize model
        use_cuda = image_features.is_cuda
        nb_batch, nb_image_feats, _ = image_features.size()

        v_mean = image_features.mean(dim=1)
        state, current_word = self.init_inference(nb_batch, use_cuda)

        # Initialize beam search
        end_word = indexto1hot(len(self.vocab), self.vocab('<end>'))
        end_word = torch.from_numpy(end_word).float().unsqueeze(0)
        end_word = end_word.cuda() if image_features.is_cuda else end_word
        beam = Beam(beam_width)
        s = Sentence(max_nb_words, beam_width, end_word, self.vocab)
        s.update_state(1.0, state, current_word)
        beam.push(s)

        # Perform beam search
        final_beam = Beam(beam_width)
        while len(beam) > 0:
            s = beam.pop()
            new_sentences = self.update_states(s, image_features, v_mean, semantic_input)
            for s in new_sentences:
                if s.ended:
                    final_beam.push(s)
                else:
                    beam.push(s)
            # Get rid of low scoring sentences
            beam.trim() 
            final_beam.trim()

        # Extract final sentence
        s = final_beam.pop() # Best sentence on top of heap
        sentence = s.extract_sentence()
        
        return sentence

    def update_states(self, s, image_feats, v_mean, semantic_input):
        state, current_word = s.get_states()
        y, state = self.forward_one_step(state, current_word, v_mean, image_feats, semantic_input)
        y = self.remove_consecutive_words(y, current_word)
        new_sentences = s.update_words(s, state, y)
        return new_sentences

    def remove_consecutive_words(self, y, prev_word):
        # give previous word low score so as not to repeat words
        y = y - 10**10 * prev_word
        return y


## Pipeline

In [15]:
def build_vocab(word_list):
    
    vocab = Vocabulary()
    
    vocab.add_word(start_token)
    vocab.add_word(end_token)
    vocab.add_word(unknown_token)
    vocab.add_word(padding_token)
    
    for word in word_list:
        vocab.add_word(word)
        
    return vocab

In [16]:
# df_word_list = pd.read_csv("data/word_list.csv")

In [17]:
# vocab = build_vocab(df_word_list.word_list)

### Load Models

In [1]:
WORD_EMB_DIM = 1000
NB_HIDDEN_LSTM1 = 1000
NB_HIDDEN_LSTM2 = 1000
NB_HIDDEN_ATT = 512

IMAGE_FEATURE_DIM = 2048
FACTORED_DIM = 1024
SEMANTIC_DIM = 1000

TF_RATIO = 1

In [19]:
# image_tagger = ImageTagger()
# image_tagger = image_tagger.cuda()
# image_tagger.load_state_dict(torch.load("models/image_tagger.best_1")['model_state_dict'])

<All keys matched successfully>

In [20]:
# encoder = image_tagger.resnet

In [21]:
# caption_model = CaptionModel(len(vocab), IMAGE_FEATURE_DIM, vocab, TF_RATIO)
# caption_model = caption_model.cuda()
# caption_model.load_state_dict(torch.load("models/caption.best_bleu3_1")['model_state_dict'])

<All keys matched successfully>

### Predict

In [48]:
# image_transformer = transforms.Compose([
#     transforms.Resize(224),
#     transforms.RandomResizedCrop(224, scale=(1.0, 1.0)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

In [2]:
def predict(caption_model, image_tagger, encoder, vocab, image_file, use_url=False):

    image_transformer = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomResizedCrop(224, scale=(1.0, 1.0)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    caption_model.eval()
    image_tagger.eval()

    if use_url:
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    
    display(image)
    
#     plt.imshow(image)
    image = image_transformer(image)
    image = image.unsqueeze(0).to(device)
    
    with torch.no_grad():
        semantic_input = image_tagger(image)
        image_feats = encoder(image)
        image_feats = image_feats.unsqueeze(1).squeeze(3).squeeze(3)
        output = caption_model(image_feats, 12, semantic_input, None, beam=5)
        
    output = output[1][1:-1] if output[1][-1] == end_token else output[1][1:]

    return ' '.join(output)

## Getter for Importer

In [1]:
def get_vocab():

    df_word_list = pd.read_csv("data/word_list.csv")
    vocab = build_vocab(df_word_list.word_list)
    
    return vocab

In [2]:
def get_models(vocab):
    image_tagger = ImageTagger()
    image_tagger = image_tagger.cuda()
    image_tagger.load_state_dict(torch.load("models/image_tagger.best_1")['model_state_dict'])
    
    encoder = image_tagger.resnet
    
    caption_model = CaptionModel(len(vocab), IMAGE_FEATURE_DIM, vocab, TF_RATIO)
    caption_model = caption_model.cuda()
    caption_model.load_state_dict(torch.load("models/caption.best_bleu3__best")['model_state_dict'])
    
    return image_tagger, encoder, caption_model

In [1]:
def predict_io():
    
    print("Loading model...")
    vocab = get_vocab()
    image_tagger, encoder, caption_model = get_models(vocab)
    
    use_url = bool(input("Use url for input? (Leave blank for False)"))

    image_file = input("\nInsert image url/file path: (Leave blank to exit)")

    while image_file != "":
        print("Generated caption:", predict(caption_model, image_tagger, encoder, vocab, image_file, use_url=use_url))

        image_file = input("\nInsert image url/file path: (Leave blank to exit)")
    
    print("\nThank you :)")